In [1]:
import langchain
print(langchain.__version__)

1.2.0


In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_community.utilities import SerpAPIWrapper
from langchain_community.document_loaders import WebBaseLoader
import dotenv
from dotenv import load_dotenv
import os
from langchain_core.tools import tool,Tool
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.llms import FakeListLLM


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
load_dotenv()
serp_api_key=os.getenv("SERP_API_KEY")
google_api_key=os.getenv("GOOGLE_API_KEY")

PROMPT TEST-1


In [4]:
prompt='''You are an expert educator and quiz master for advanced technical students.
Your goal is to create a quiz based strictly on the following text context.
Text Context:   "{text}"

Instructions to be followed:
1. Create {num_questions} multiple-choice questions(MCQs) based only on the Text above"
2. For each question, provide 4 options (A, B, C, D)
3. Identfiy the correct answer
4. Ensure the questions test understanding, not just simple words matching.
5. Difficulty level : {difficulty} (eg. easy,medium or hard)

Output Format: 
You must output a strictly valid JSON string. Do not output markdown code blocks.
The JSON format must be list of objects like this: 
[
    {{
        "question": "The question text here?",
        "options": ["Option A", "Option B", "Option C", "Option D"],
        "correct_answer": "Option B",
        "explanation": "A brief explanation of why B is correct."
    }}
]
'''

In [5]:
prompt_template=PromptTemplate(input_variables=["text","num_questions","difficulty"],template=prompt)

In [6]:
final_prompt=(prompt_template.format(text='''International Debut: He made his ODI debut for India on August 18, 2008, against Sri Lanka.
World Cup Success: Kohli was a key member of the Indian team that won the 2011 ODI World Cup, the 2013 ICC Champions Trophy, and the 2024 T20 World Cup.
Most ODI Centuries: He holds the world record for the most centuries in One Day Internationals (ODIs), surpassing Sachin Tendulkar's long-standing record with his 50th ODI century during the 2023 World Cup.
Record Run Scorer: He is the fastest batsman to reach multiple thousand-run milestones in ODIs, including 8,000, 10,000, 12,000, and 14,000 runs.
Successful Test Captain: As India's most successful Test captain, he led the team to a historic Test series win in Australia in 2018-19 and helped them maintain the #1 Test ranking for a long period.
ICC Awards: His numerous accolades include the Sir Garfield Sobers Trophy (ICC Cricketer of the Year) in 2017 and 2018, and the ICC Men's ODI Cricketer of the Decade (2011–2020).
IPL Career: In the Indian Premier League (IPL), he has played exclusively for the Royal Challengers Bengaluru (RCB) since 2008 and led them to their first title in the 2025 season.
Retirement Announcements: Kohli retired from T20 International cricket in 2024 after India's T20 World Cup victory and announced his retirement from Test cricket in May 2025.
Personal Life: He is married to popular Bollywood actress Anushka Sharma, and they have two children together, a daughter named Vamika and a son named Akaay.
Off-Field Influence: Known for his discipline and passion, he is a role model for aspiring athletes and a highly influential global figure with a significant net worth from endorsements and business ventures.'''
, num_questions=10,difficulty='easy'))

USINNG  MOCK  LLM FOR TESTING PURPOSES

In [ ]:


USE_MOCK = True

dummy_response_json = """
[
    {
        "question": "Who holds the record for the most ODI centuries?",
        "options": ["Sachin Tendulkar", "Virat Kohli", "Ricky Ponting", "Rohit Sharma"],
        "correct_answer": "Virat Kohli",
        "explanation": "Virat Kohli surpassed Sachin Tendulkar's record with his 50th ODI century during the 2023 World Cup."
    },
    {
        "question": "Which IPL team has Virat Kohli played for exclusively since 2008?",
        "options": ["Mumbai Indians", "Chennai Super Kings", "Royal Challengers Bengaluru", "Kolkata Knight Riders"],
        "correct_answer": "Royal Challengers Bengaluru",
        "explanation": "He has played exclusively for RCB since the inception of the IPL in 2008."
    }
]
"""

if USE_MOCK:
    print("Using FAKE LLM for testing...")
    # FakeListLLM expects a list of responses to cycle through
    client = FakeListLLM(responses=[dummy_response_json])
    response = client.invoke(final_prompt) 
    # Note: FakeListLLM returns string directly in some versions, or object in others. 
    # Invoke typically returns a string for legacy LLMs or BaseMessage for ChatModels.
    # Let's handle both just in case.
    if hasattr(response, 'content'):
        print(response.content)
    else:
        print(response)
else:
    print("Using REAL Gemini LLM...")
    client = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
    response = client.invoke(final_prompt)
    print(response.content)

Using FAKE LLM for testing...

[
    {
        "question": "Who holds the record for the most ODI centuries?",
        "options": ["Sachin Tendulkar", "Virat Kohli", "Ricky Ponting", "Rohit Sharma"],
        "correct_answer": "Virat Kohli",
        "explanation": "Virat Kohli surpassed Sachin Tendulkar's record with his 50th ODI century during the 2023 World Cup."
    },
    {
        "question": "Which IPL team has Virat Kohli played for exclusively since 2008?",
        "options": ["Mumbai Indians", "Chennai Super Kings", "Royal Challengers Bengaluru", "Kolkata Knight Riders"],
        "correct_answer": "Royal Challengers Bengaluru",
        "explanation": "He has played exclusively for RCB since the inception of the IPL in 2008."
    }
]



In [23]:
messages=client.invoke(prompt.format(text='Hello sir how are you',num_questions=2,difficulty='easy'))

In [24]:
print(messages)


[
    {
        "question": "Who holds the record for the most ODI centuries?",
        "options": ["Sachin Tendulkar", "Virat Kohli", "Ricky Ponting", "Rohit Sharma"],
        "correct_answer": "Virat Kohli",
        "explanation": "Virat Kohli surpassed Sachin Tendulkar's record with his 50th ODI century during the 2023 World Cup."
    },
    {
        "question": "Which IPL team has Virat Kohli played for exclusively since 2008?",
        "options": ["Mumbai Indians", "Chennai Super Kings", "Royal Challengers Bengaluru", "Kolkata Knight Riders"],
        "correct_answer": "Royal Challengers Bengaluru",
        "explanation": "He has played exclusively for RCB since the inception of the IPL in 2008."
    }
]



Testing PyPDF

In [7]:
from langchain_community.document_loaders import PyPDFLoader

loader=PyPDFLoader(r"C:\ALL PROJECTS\BrainBolt\data\pdfs\AI_Powered_Document_Search_RAG_Overview.pdf")
pages=loader.load()
full_text="\n".join([page.page_content for page in pages])
print(full_text)



AI-Powered Document Search & Question Answering (RAG System)
Purpose & Vision
This project aims to develop an AI-powered system that enables users to upload, search, and
query documents with intelligent, context-aware responses. The system integrates Retrieval-
Augmented Generation (RAG) with Google Gemini, combining local vector search with a generative
language model to produce accurate and grounded answers.
Core Concept: Retrieval-Augmented Generation (RAG)
Retrieval-Augmented Generation (RAG) bridges the gap between retrieval-based search and
generative AI. It retrieves the most relevant document chunks through vector similarity search
and feeds them to a large language model (LLM), ensuring responses are contextually accurate,
citeable, and based on real data rather than hallucinations. This creates a balance between
factual precision and generative reasoning.
System Architecture
1. Document ingestion: Users can upload PDF, DOCX, TXT, or provide URLs.
2. Text extraction: Raw text 

In [4]:
import os
print(os.path.isfile("C:\ALL PROJECTS\BrainBolt\data\pdfs\AI_Powered_Document_Search_RAG_Overview.pdf"))


True


Testing the search ingestor


In [5]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper


In [6]:
import sys
import logging
import os

In [7]:
sys.path.append(os.path.abspath('..'))
logging.basicConfig(level=logging.INFO)

In [12]:
from src.ingestors.search import SearchIngestor

In [13]:

ingestor = SearchIngestor(num_results=3)
res = ingestor.load("List the top 5 nulcear nations in the world")
print(res)


INFO:src.ingestors.search:Searching the WEB for List the top 5 nulcear nations in the world
INFO:ddgs.ddgs:Error in engine grokipedia: DDGSException("RuntimeError: RuntimeError('error sending request for url (https://wt.wikipedia.org/w/api.php?action=opensearch&profile=fuzzy&limit=1&search=List%20the%20top%205%20nulcear%20nations%20in%20the%20world): client error (Connect)\\n\\nCaused by:\\n    0: client error (Connect)\\n    1: dns error: No such host is known. (os error 11001)\\n    2: No such host is known. (os error 11001)')")
INFO:primp:response: https://grokipedia.com/api/typeahead?query=List+the+top+5+nulcear+nations+in+the+world&limit=1 200
INFO:primp:response: https://www.mojeek.com/search?q=List+the+top+5+nulcear+nations+in+the+world 200
INFO:src.ingestors.search:Found['https://desultoryheroics.com/tag/nulcear-war/', 'https://newsoftheworld.us/iran-top-diplomat-hopes-for-restoration-of-saudi-ties', 'https://newsoftheworld.us/gaza-no-longer-classed-as-in-famine'] for List the 












Nulcear War | After the Crash 	



















































 








Skip to primary contentSkip to secondary content


After the Crash 
Dispatches From a Long Recovery (Est. 10/2024)





Search




Main menu

HomeAbout
Archives
RSS
Support
Contact







Tag Archives: Nulcear War



The Elite’s COVID-19 Coup to Destroy Humanity that is also Fast-Tracking Four Paths to Human Extinction

Posted on June 12, 2020  by  Reid Mukai 

1 



By Robert J. Burrowes
The global elite is conducting a coup that is designed to destroy all of the key elements of human society. It is doing this by destroying the essence of what it means to be human, by destroying the nature of existing human relationships, and by destroying the political, economic and social institutions of nation states.
Intentionally or otherwise, the elite coup is also fast-tracking four paths to human extinction.
If this coup succeeds, the human individual will have been reduced to a digitized id

In [15]:
pip install -U ddgs

  Using cached ddgs-9.10.0-py3-none-any.whl.metadata (12 kB)
  Using cached primp-0.15.0-cp38-abi3-win_amd64.whl.metadata (13 kB)
  Using cached lxml-6.0.2-cp311-cp311-win_amd64.whl.metadata (3.7 kB)
  Using cached fake_useragent-2.2.0-py3-none-any.whl.metadata (17 kB)
  Using cached brotli-1.2.0-cp311-cp311-win_amd64.whl.metadata (6.3 kB)
  Using cached h2-4.3.0-py3-none-any.whl.metadata (5.1 kB)
  Using cached socksio-1.0.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached hyperframe-6.1.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached hpack-4.1.0-py3-none-any.whl.metadata (4.6 kB)
Using cached ddgs-9.10.0-py3-none-any.whl (40 kB)
Using cached fake_useragent-2.2.0-py3-none-any.whl (161 kB)
Using cached h2-4.3.0-py3-none-any.whl (61 kB)
Using cached hpack-4.1.0-py3-none-any.whl (34 kB)
Using cached hyperframe-6.1.0-py3-none-any.whl (13 kB)
Using cached socksio-1.0.0-py3-none-any.whl (12 kB)
Using cached lxml-6.0.2-cp311-cp311-win_amd64.whl (4.0 MB)
Using cached primp-0.15.0-cp38-ab

QUIZ Generator


In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel,Field
from typing import List

In [15]:
logger=logging.getLogger(__name__)

In [16]:
class QuizQuestion(BaseModel):
    question : str =Field(description="The text")
    options :List[str]=Field(description="THe options")
    correct_answer :int=Field(description="The correct answer")
    explanation :str=Field(description="The explanation")

In [17]:
class QuizOutput(BaseModel):
    quiz: List[QuizQuestion]

In [18]:
llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash",temperature=0.3)
parser=JsonOutputParser(pydantic_object=QuizOutput)

PROMPT Template to be used


In [19]:
prompt = PromptTemplate(
    template="""You are an expert exam setter. Create {num_questions} {difficulty} level multiple-choice questions based on the following text.
    
    Text: "{text}"
    
    {format_instructions}
    """,
    input_variables=["num_questions", "difficulty", "text"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [20]:
chain=prompt | llm |parser


In [21]:
import json
if res in locals():
    print("Genration in progress")
    try:
        quiz_data = chain.invoke({
            "num_questions": 3,
            "difficulty": "Easy",
            "text": res  
        })
        print(json.dumps(quiz_data, indent=2))
    except Exception as e:
        print(f"Error during generation: {e}")
else:
    print("Please define res")

        

Please define res


In [21]:
response = chain.invoke({
    "text": "Your search result text goes here...", 
    "num_questions": 5, 
    "difficulty": "Easy"
})

print(response)

INFO:google_genai.models:AFC is enabled with max remote calls: 10.
INFO:httpx:HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"


{'quiz': [{'question': 'What is the primary purpose of the text provided?', 'options': ['To provide a recipe for baking cookies', 'To explain the process of photosynthesis', 'To offer a general introduction to a topic', 'To detail the history of the Roman Empire'], 'correct_answer': 2, 'explanation': "The text is described as 'Your search result text goes here...', implying it's a placeholder for general information that would be found in search results."}, {'question': 'Based on the placeholder text, what kind of content is expected?', 'options': ['A fictional story', 'A technical manual', 'Informational content', 'A personal diary entry'], 'correct_answer': 2, 'explanation': "Search results typically contain informational content that answers a user's query."}, {'question': "If the text were about 'how to tie a knot', what would be the most likely format?", 'options': ['A song', 'A poem', 'Instructions or a guide', 'A mathematical equation'], 'correct_answer': 2, 'explanation': 'Inst

In [22]:
print(response)

{'quiz': [{'question': 'What is the primary purpose of the text provided?', 'options': ['To provide a recipe for baking cookies', 'To explain the process of photosynthesis', 'To offer a general introduction to a topic', 'To detail the history of the Roman Empire'], 'correct_answer': 2, 'explanation': "The text is described as 'Your search result text goes here...', implying it's a placeholder for general information that would be found in search results."}, {'question': 'Based on the placeholder text, what kind of content is expected?', 'options': ['A fictional story', 'A technical manual', 'Informational content', 'A personal diary entry'], 'correct_answer': 2, 'explanation': "Search results typically contain informational content that answers a user's query."}, {'question': "If the text were about 'how to tie a knot', what would be the most likely format?", 'options': ['A song', 'A poem', 'Instructions or a guide', 'A mathematical equation'], 'correct_answer': 2, 'explanation': 'Inst

PaddleOCR

Resolved by creating a seperate isolated environment for paddleocr to avoid langchain conflicts

The code for the same is below and please do not run since it uses a seperate venv 


In [ ]:
from paddleocr import PaddleOCR
import logging

In [ ]:
logging.getLogger("ppocr").setLevel(logging.ERROR)
image_path="Enter your path here"

In [ ]:
ocr=PaddleOCR(use_angle_cls=True,lang="en")
result=ocr.ocr(image_path,cls=True)

In [ ]:
if result and result[0]:
  for line in result[0]:
    text_conent=line[1][0]
    confidence=line[1][1]
    print(f"Text:{text_conent},Confidence:{confidence}")
else:
  print("No text")

Demo Output for the same 


Text:TEXTUAL,Confidence:0.9988256096839905

Text:AIDS,Confidence:0.9899177551269531

Text:ortext,Confidence:0.8826766610145569

Text:features are visual elements,Confidence:0.9981244802474976

Text:on a page such as titles,,Confidence:0.9997007846832275

Text:bold or italic prints, charts,,Confidence:0.9880038499832153

Text:diagrams, maps and tables,,Confidence:0.974068284034729

Text:written,Confidence:0.912692129611969

Text:texts,prints,Confidence:0.944272518157959

Text:and,Confidence:0.8644866347312927

Text:some other way of writing,Confidence:0.9994122982025146

Text:for an important word,Confidence:0.9707242250442505

Text:to,Confidence:0.9977688789367676

Text:stand out from the other.,Confidence:0.9895346164703369

OCR Testing


In [16]:
from src.ingestors.base import BaseIngestor
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
import base64


In [20]:
llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash")

def encode_image(image_path):
    with open(image_path,"rb") as image_file:
        image_bytes=image_file.read()
        encoded_bytes=base64.b64encode(image_bytes)
        encoded_string=encoded_bytes.decode("utf-8")
        return encoded_string

Enter the Image  Path here

In [21]:
image_path="C:\ALL PROJECTS\BrainBolt\data\ocr_testing_image.jpg"


In [22]:
image_data=encode_image(image_path)
message=HumanMessage(
        content=[
            {"type": "text", "text": "Extract all text from this image and provide a comprehensive summary of the visual content suitable for generating quiz questions."},
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image_data}"}
            }
        ]
    )